### Try to use LinearMap with ismutating=true, fail

In [ ]:
using LinearMaps
using IterativeSolvers

In [ ]:
function g(y::AbstractVector,x::AbstractVector) 
    y = [1 2;3 5]*x
    return y
end

S = LinearMap(g,2;ismutating=true)
b = [4.0, 3.3]
x = [2.0, 2.0]
gmres!(x,S,b)

### Try to use LinearMap with ismutating=false, works

In [ ]:
function f(x::AbstractVector) 
    return [1 2;3 5]*x
end

S = LinearMap(f,2;ismutating=false)
b = [4.0, 3.3]
gmres(S,b)

In [ ]:
[1 2;3 5]\[4.,3.3]

### Construct LinearMap with AbstractMatrix type

In [ ]:
[1 2;3 5]*[1 1.5;2 2]

In [ ]:
function f(x::AbstractMatrix,b::Int,c::Float64) 
    return [1 2;3 5]*x
end

S = LinearMap(f([1 1.5;2 2],999,-888.0))
b = [4.0, 3.3]
gmres(S,b)

In [ ]:
[5 5.5;13 14.5]\[4.,3.3]

### Construct block matrix with functions, works

In [ ]:
m1 = [ 0.533442  0.367231  0.217876;
 0.937095  0.838996  0.38914;
 0.399906  0.409811  0.571862]
m2 = [ 0.515455  0.912171;
 0.197663  0.165777;
 0.3626    0.735575]
m3 = [ 0.763043  0.36013   0.779911; 
 0.676889  0.262959  0.0735239]
m4 = [ 0.981832  0.695422;
 0.906051  0.432737]
b = [ 0.71537 
 0.37217 
 0.728864
 0.72988 
 0.881618]

[m1 m2;m3 m4]\b

In [ ]:
function al(x::AbstractVector)

m1 = [ 0.533442  0.367231  0.217876;
 0.937095  0.838996  0.38914;
 0.399906  0.409811  0.571862]
m2 = [ 0.515455  0.912171;
 0.197663  0.165777;
 0.3626    0.735575]
m3 = [ 0.763043  0.36013   0.779911; 
 0.676889  0.262959  0.0735239]
m4 = [ 0.981832  0.695422;
 0.906051  0.432737]

    
    function f(x::AbstractVector) 
        return m1*x
    end
    function g(x::AbstractVector) 
        return m2*x
    end
    function m(x::AbstractVector) 
        return m3*x
    end
    function n(x::AbstractVector) 
        return m4*x
    end 
    
    # has to do the matrix mul manually to return 1d output
    out = zeros(5)
    out[1:3] = f(x[1:3]) + g(x[4:5])
    out[4:5] = m(x[1:3]) + n(x[4:5])
    
    return out
end
S = LinearMap(al,5;ismutating=false)    
    
b = [ 0.71537 
 0.37217 
 0.728864
 0.72988 
 0.881618]
gmres(S,b)

### Construct block matrix with mixed LinearMap and FunctionMap,  works

In [ ]:
m1 = [ 0.533442  0.367231  0.217876;
 0.937095  0.838996  0.38914;
 0.399906  0.409811  0.571862]
m2 = [ 0.515455  0.912171;
 0.197663  0.165777;
 0.3626    0.735575]
m3 = [ 0.763043  0.36013   0.779911; 
 0.676889  0.262959  0.0735239]
m4 = [5 5.5;13 14.5]
b = [ 0.71537 
 0.37217 
 0.728864
 0.72988 
 0.881618]

[m1 m2;m3 m4]\b

In [ ]:
function al(x::AbstractVector)

m1 = [ 0.533442  0.367231  0.217876;
 0.937095  0.838996  0.38914;
 0.399906  0.409811  0.571862]
m2 = [ 0.515455  0.912171;
 0.197663  0.165777;
 0.3626    0.735575]
m3 = [ 0.763043  0.36013   0.779911; 
 0.676889  0.262959  0.0735239]

    
    function f(x::AbstractVector) 
        return m1*x
    end
    function g(x::AbstractVector) 
        return m2*x
    end
    function m(x::AbstractVector) 
        return m3*x
    end
    function n(x::AbstractMatrix,b::Int,c::Float64) 
        return [1 2;3 5]*x
    end
    
    # has to do the matrix mul manually to return 1d output
    out = zeros(5)
    out[1:3] = f(x[1:3]) + g(x[4:5])
    out[4:5] = m(x[1:3]) + n([1 1.5;2 2],999,-888.0)*x[4:5]
    
    return out
end
S = LinearMap(al,5;ismutating=false)    
    
b = [ 0.71537 
 0.37217 
 0.728864
 0.72988 
 0.881618]
gmres(S,b)

### Test SaddleSystem

In [1]:
include("../src/SaddlePointSystems.jl")
using SaddlePointSystems

In [2]:
A = rand(3,3)
A⁻¹ = inv(A)
B₁ᵀ = rand(3,2)
B₂ = rand(2,3)
M = rand(2,2)
M⁻¹ = inv(M)
G₁ᵀ = rand(2,2)
G₂ = rand(2,2)
T₁ᵀ = rand(2,2)
T₂ = rand(2,2)

O32 = zeros(3,2)
O23 = zeros(2,3)
O22 = zeros(2,2)

S = [A O32 B₁ᵀ O32; O23 M T₁ᵀ G₁ᵀ; B₂ -T₂ O22 O22; O23 G₂ O22 O22]

rċ = rand(3)
ru̇ = rand(2)
rf = rand(2)
rλ = rand(2)
b = [rċ;ru̇;rf;rλ]

S\b

9-element Array{Float64,1}:
   0.273572
  -0.405095
   1.88694 
   0.895499
   0.214429
   1.22461 
  -0.930456
   1.5315  
 -41.7307  

In [3]:
rank(S) == 3+2+2+2

true

In [4]:
ċ = zeros(3)
u̇ = zeros(2)
f = zeros(2)
λ = zeros(2)

St = SaddleSystem((ċ, u̇, f, λ), (A⁻¹, B₁ᵀ, B₂),
                  (M⁻¹, G₁ᵀ, G₂), (x->T₁ᵀ*x, x->T₂*x))

bt = (rċ, ru̇, rf, rλ);

In [5]:
aa,bb,cc,dd = St\bt

([0.273572, -0.405095, 1.88694], [0.895499, 0.214429], [1.22461, -0.930456], [1.5315, -41.7307])

In [6]:
x = [aa;bb;cc;dd]
norm(S*x-b)

2.120300723488013e-14